In [ ]:
import openai
import json
from tqdm import tqdm
from collections import defaultdict
import sys
openai.api_key_path = '<openai api key?'

In [20]:
def make_gpt_request(prompt: str, max_tokens=100):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "You are a helpful assistant, useful for code generation."},
                {"role": "user", "content": prompt},
            ],
        stop=None,
        max_tokens=max_tokens
        )
    return response

In [21]:
make_gpt_request('How are you?')

<OpenAIObject chat.completion id=chatcmpl-8MeFkimi50fdpRFkfZfchjMCnliGf at 0x7fdf1119dc60> JSON: {
  "id": "chatcmpl-8MeFkimi50fdpRFkfZfchjMCnliGf",
  "object": "chat.completion",
  "created": 1700408640,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Thank you for asking! As an AI assistant, I don't have feelings, but I'm here and ready to assist you with any coding questions or tasks you have. How can I help you today?"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 26,
    "completion_tokens": 41,
    "total_tokens": 67
  }
}

In [22]:
errors = json.load(open('../data/self_correction/responses_message_error.json', 'r'))
idx = 1
query = errors[idx]['answer_generation']['query']
error_chain_example = errors[idx]['trys'][0]['chain'][:6]
tool_descriptions = str(errors[idx]['answer_generation']['function'])
errors[idx]

{'win': False,
 'try_count': 1,
 'trys': [{'chain': [{'is_terminal': False,
     'pruned': False,
     'finished': False,
     'depth': 1,
     'node_type': 'Thought',
     'description': '',
     'Elo': 1000.0,
     'child_count': 1,
     'expand_num': 0},
    {'is_terminal': False,
     'pruned': False,
     'finished': False,
     'depth': 2,
     'node_type': 'Action',
     'description': 'get_distance_by_city_state_country_for_great_circle_math_api',
     'Elo': 1000.0,
     'child_count': 1,
     'expand_num': 0},
    {'is_terminal': False,
     'pruned': False,
     'finished': False,
     'depth': 3,
     'node_type': 'Action Input',
     'description': '{\n  "country1": "US",\n  "state1": "CA",\n  "city1": "Escondido",\n  "country2": "US",\n  "state2": "CA",\n  "city2": "Santa Rosa"\n}',
     'Elo': 1000.0,
     'observation': '{"error": "", "response": "{\'cityName1\': \'Escondido\', \'stateName1\': \'California\', \'country1\': \'US\', \'latitude1\': 33.1216751, \'longitude1

In [23]:
prompt = ''
for item in error_chain_example:
    if item['node_type'] != 'Action Input':
        prompt += item['node_type'] + ': ' + item['description'] + '\n'
    else:
        prompt += item['node_type'] + ': ' + item['description'] + '\n' + 'Observation: ' + item['observation'] + '\n'

request = f'The user asked: {query}\n'
tool_descriptions_prompt = f'Here are tool descriptions: {tool_descriptions}\n'
initial_prompt = 'You are given an example of a tool chain by a model called Toollama and a response from a tool.\nExample begins\n'
final_prompt = tool_descriptions_prompt + request + initial_prompt + prompt + 'Example ends\nAnswer, if this call resulted in an error. If so, explain an error and provide short explanation of how to solve it. Do not write any code. Do not make up anything. Explain an error based on tool description, tool parameters format (in the field "parameters"), on response from tool and on your knowledge. Make your response short, just single sentence.\n'
print(final_prompt)

Here are tool descriptions: [{'name': 'get_distance_by_city_state_country_for_great_circle_math_api', 'description': 'This is the subfunction for tool "great_circle_math_api", you can use this tool.The description of this function is: "Takes city, state, and country of both locations and returns latitude, longitude, and calculated miles."', 'parameters': {'type': 'object', 'properties': {'country1': {'type': 'string', 'description': '', 'example_value': 'us'}, 'country2': {'type': 'string', 'description': '', 'example_value': 'us'}, 'state2': {'type': 'string', 'description': '', 'example_value': 'ca'}, 'city2': {'type': 'string', 'description': '', 'example_value': 'sacramento'}, 'city1': {'type': 'string', 'description': '', 'example_value': 'birmingham'}, 'state1': {'type': 'string', 'description': '', 'example_value': 'al'}}, 'required': ['country1', 'country2', 'state2', 'city2', 'city1', 'state1'], 'optional': []}}, {'name': 'm_one_unit_of_measure_to_another_for_measurement_units

In [24]:
make_gpt_request(final_prompt)

<OpenAIObject chat.completion id=chatcmpl-8MeFndOohUIJaFFOJx6QN13unbkY1 at 0x7fdee4618540> JSON: {
  "id": "chatcmpl-8MeFndOohUIJaFFOJx6QN13unbkY1",
  "object": "chat.completion",
  "created": 1700408643,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The error occurred because the measurement_units_converter tool does not support conversion from miles to miles. To solve this, you can choose a different unit as the output_unit."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 1051,
    "completion_tokens": 33,
    "total_tokens": 1084
  }
}

Попробуем с тем промптом, который предлагается в статье MINT

In [25]:
mint_prompt = '''
You are an expert tasked with evaluating and providing feedback on an assistant's performance.

---
Here is an example. Please follow the format as the following expert acts.

Example:

Example tool descriptions:
{example_tool_desc}

Example assistant's trajectory:
{example_trajectory}

Example expert feedback:
{feedback}

---

Tool descriptions:
{tool_desc}

Assistant's trajectory
{trajectory}

Please provide concise and constructive feedback. Remember, your role is similar to a teacher. Rather than giving away the solution or details about the answer, guide the assistant toward understanding how to arrive at the correct answer. Your feedback should focus on enhancing the assistant's ability to think critically and respond accurately. Now provide your feedback.
Expert feedback:'''

Составим in context learning пример
Его лучше составить из пример а